In [1]:
import boto3
import os

In [2]:
access_key_id = "minioadmin"
secret_access_key = "minioadmin"
minio_url = "http://localhost:9000"

minio_client = boto3.client(
    "s3",
    aws_access_key_id=access_key_id,
    aws_secret_access_key=secret_access_key,
    endpoint_url=minio_url
)

In [ ]:
from pydub import AudioSegment
import io
from tqdm import tqdm

bucket_origen = "persistent-landing"
bucket_desti = "formatted-zone"
prefix_origen = "audios/"
freq_final = 44100

paginator = minio_client.get_paginator("list_objects_v2")

for page in paginator.paginate(Bucket=bucket_origen, Prefix=prefix_origen):
    for obj in tqdm(page.get("Contents", []), desc="Processant àudios"):
        key = obj["Key"]
        filename = key.split("/")[-1]

        if not filename.lower().endswith(".mp3"):
            continue

        #Llegir àudio original
        response = minio_client.get_object(Bucket=bucket_origen, Key=key)
        audio_data = response["Body"].read()

        #Obrir amb pydub
        try:
            audio = AudioSegment.from_file(io.BytesIO(audio_data), format="mp3")
        except Exception as e:
            print(f"⚠️ Error amb {filename}: {e}")
            continue

        #Normalitzar freqüència i volum
        audio = audio.set_frame_rate(freq_final)
        audio = audio.normalize()  # opcional: volum uniforme

        #Desa com MP3 i puja al bucket formatted-zone
        buffer = io.BytesIO()
        audio.export(buffer, format="mp3")
        buffer.seek(0)

        new_key = f"audios/{filename}"
        minio_client.put_object(
            Bucket=bucket_desti,
            Key=new_key,
            Body=buffer
        )